In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout


2024-07-15 17:23:22.445379: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:

train_df = pd.read_csv('/home/amritpal-singh/Downloads/train_FD001.txt', sep='\s+', header=None)
test_df = pd.read_csv('/home/amritpal-singh/Downloads/test_FD001.txt', sep='\s+', header=None)
rul_df = pd.read_csv('/home/amritpal-singh/Downloads/RUL_FD001.txt', header=None)


columns = ['id', 'cycle', 'setting1', 'setting2', 'setting3'] + [f'sensor_{i}' for i in range(1, 22)]
train_df.columns = columns
test_df.columns = columns

print(train_df.head())

   id  cycle  setting1  setting2  setting3  sensor_1  sensor_2  sensor_3  \
0   1      1   -0.0007   -0.0004     100.0    518.67    641.82   1589.70   
1   1      2    0.0019   -0.0003     100.0    518.67    642.15   1591.82   
2   1      3   -0.0043    0.0003     100.0    518.67    642.35   1587.99   
3   1      4    0.0007    0.0000     100.0    518.67    642.35   1582.79   
4   1      5   -0.0019   -0.0002     100.0    518.67    642.37   1582.85   

   sensor_4  sensor_5  ...  sensor_12  sensor_13  sensor_14  sensor_15  \
0   1400.60     14.62  ...     521.66    2388.02    8138.62     8.4195   
1   1403.14     14.62  ...     522.28    2388.07    8131.49     8.4318   
2   1404.20     14.62  ...     522.42    2388.03    8133.23     8.4178   
3   1401.87     14.62  ...     522.86    2388.08    8133.83     8.3682   
4   1406.22     14.62  ...     522.19    2388.04    8133.80     8.4294   

   sensor_16  sensor_17  sensor_18  sensor_19  sensor_20  sensor_21  
0       0.03        392     

/tmp/ipykernel_13525/2057557527.py:1: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  train_df = pd.read_csv('/home/amritpal-singh/Downloads/train_FD001.txt', delim_whitespace=True, header=None)
/tmp/ipykernel_13525/2057557527.py:2: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  test_df = pd.read_csv('/home/amritpal-singh/Downloads/test_FD001.txt', delim_whitespace=True, header=None)


In [4]:
scaler = MinMaxScaler()
train_df.iloc[:, 2:] = scaler.fit_transform(train_df.iloc[:, 2:]).astype(float)
test_df.iloc[:, 2:] = scaler.transform(test_df.iloc[:, 2:]).astype(float)

train_df['RUL'] = train_df.groupby('id')['cycle'].transform(lambda x: x.max() - x)

/tmp/ipykernel_13525/3011002836.py:2: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33333333 0.33333333 0.16666667 ... 0.83333333 0.58333333 0.66666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.iloc[:, 2:] = scaler.fit_transform(train_df.iloc[:, 2:])
/tmp/ipykernel_13525/3011002836.py:3: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33333333 0.41666667 0.41666667 ... 0.58333333 0.58333333 0.66666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_df.iloc[:, 2:] = scaler.transform(test_df.iloc[:, 2:])


In [8]:

def create_sequences(data, seq_length=50):
    sequences = []
    for id in data['id'].unique():
        engine_data = data[data['id'] == id]
        for i in range(len(engine_data) - seq_length + 1):
            seq = engine_data.iloc[i:i+seq_length].drop(columns=['id', 'cycle', 'RUL']).values
            label = engine_data.iloc[i+seq_length-1]['RUL']
            sequences.append((seq, label))
    return sequences

train_sequences = create_sequences(train_df)

X_train = np.array([seq[0] for seq in train_sequences])
y_train = np.array([seq[1] for seq in train_sequences])

In [9]:
test_df

,id,cycle,setting1,setting2,setting3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_12,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21
0,1,1,0.632184,0.750000,0.0,0.0,0.545181,0.310661,0.269413,0.0,...,0.646055,0.220588,0.132160,0.308965,0.0,0.333333,0,0.0,0.558140,0.661834
1,1,2,0.344828,0.250000,0.0,0.0,0.150602,0.379551,0.222316,0.0,...,0.739872,0.264706,0.204768,0.213159,0.0,0.416667,0,0.0,0.682171,0.686827
2,1,3,0.517241,0.583333,0.0,0.0,0.376506,0.346632,0.322248,0.0,...,0.699360,0.220588,0.155640,0.458638,0.0,0.416667,0,0.0,0.728682,0.721348
3,1,4,0.741379,0.500000,0.0,0.0,0.370482,0.285154,0.408001,0.0,...,0.573561,0.250000,0.170090,0.257022,0.0,0.250000,0,0.0,0.666667,0.662110
4,1,5,0.580460,0.500000,0.0,0.0,0.391566,0.352082,0.332039,0.0,...,0.737740,0.220588,0.152751,0.300885,0.0,0.166667,0,0.0,0.658915,0.716377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13091,100,194,0.781609,0.500000,0.0,0.0,0.611446,0.619359,0.566172,0.0,...,0.426439,0.176471,0.584890,0.564063,0.0,0.500000,0,0.0,0.395349,0.418669
13092,100,195,0.436782,0.416667,0.0,0.0,0.605422,0.537388,0.671843,0.0,...,0.503198,0.308824,0.572350,0.485956,0.0,0.583333,0,0.0,0.333333,0.528721
13093,100,196,0.465517,0.250000,0.0,0.0,0.671687,0.482014,0.414754,0.0,...,0.530917,0.235294,0.605326,0.507888,0.0,0.583333,0,0.0,0.372093,0.429301
13094,100,197,0.281609,0.583333,0.0,0.0,0.617470,0.522128,0.626435,0.0,...,0.562900,0.294118,0.622046,0.562524,0.0,0.583333,0,0.0,0.403101,0.518779


In [10]:
from sklearn.metrics import mean_squared_error
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

test_sequences = create_sequences(test_df)
X_test = np.array([seq[0] for seq in test_sequences])

y_pred = model.predict(X_test)

rmse = np.sqrt(mean_squared_error(rul_df, y_pred))
print(f'Test RMSE: {rmse}')

predictions_df = pd.DataFrame(y_pred, columns=['Predicted_RUL'])
predictions_df.to_csv('predicted_rul.csv', index=False)

Epoch 1/100


2024-07-15 18:16:07.509899: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 18:16:07.510559: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 18:16:07.511222: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

304/394 [======================>.......] - ETA: 2s - loss: 7343.6299

KeyboardInterrupt: 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense

# Suppress TensorFlow informational messages
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Column names based on the provided data structure
column_names = [
    'id', 'cycle', 'setting1', 'setting2', 'setting3', 's1', 's2', 's3', 's4', 's5',
    's6', 's7', 's8', 's9', 's10', 's11', 's12', 's13', 's14', 's15', 's16', 's17', 's18', 's19', 's20', 's21'
]

# Load the datasets
train_df = pd.read_csv('/home/amritpal-singh/Downloads/train_FD001.txt', sep='\s+', header=None, names=column_names)
test_df = pd.read_csv('/home/amritpal-singh/Downloads/test_FD001.txt', sep='\s+', header=None, names=column_names)
rul_df = pd.read_csv('/home/amritpal-singh/Downloads/RUL_FD001.txt', header=None)

# Add RUL column to train data
train_df['RUL'] = train_df.groupby('id')['cycle'].transform(max) - train_df['cycle']

# Initialize the scaler
scaler = MinMaxScaler()

# Transform the data
train_df.iloc[:, 2:-1] = scaler.fit_transform(train_df.iloc[:, 2:-1].astype(float))
test_df.iloc[:, 2:] = scaler.transform(test_df.iloc[:, 2:].astype(float))

# Define the create_sequences function
def create_sequences(data, seq_length=50):
    sequences = []
    for id in data['id'].unique():
        engine_data = data[data['id'] == id]
        for i in range(len(engine_data) - seq_length + 1):
            seq = engine_data.iloc[i:i + seq_length].drop(columns=['id', 'cycle', 'RUL']).values
            label = engine_data.iloc[i + seq_length - 1]['RUL']
            sequences.append((seq, label))
    return sequences

# Create sequences for training
sequence_length = 50
train_sequences = create_sequences(train_df, sequence_length)
X_train = np.array([seq[0] for seq in train_sequences])
y_train = np.array([seq[1] for seq in train_sequences])

# Define the LSTM model
model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(50, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1))

# Compile and train the model
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2)

# Create sequences for testing
test_sequences = create_sequences(test_df, sequence_length)
X_test = np.array([seq[0] for seq in test_sequences])

# Predict RUL for test data
y_pred = model.predict(X_test)

# Compute RMSE
rmse = np.sqrt(mean_squared_error(rul_df, y_pred))
print(f'Test RMSE: {rmse}')

# Save predictions to CSV
predictions_df = pd.DataFrame(y_pred, columns=['Predicted_RUL'])
predictions_df.to_csv('predicted_rul.csv', index=False)


/tmp/ipykernel_13525/2404001587.py:24: FutureWarning: The provided callable <built-in function max> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  train_df['RUL'] = train_df.groupby('id')['cycle'].transform(max) - train_df['cycle']
/tmp/ipykernel_13525/2404001587.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33333333 0.33333333 0.16666667 ... 0.83333333 0.58333333 0.66666667]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_df.iloc[:, 2:-1] = scaler.fit_transform(train_df.iloc[:, 2:-1].astype(float))
/tmp/ipykernel_13525/2404001587.py:31: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.33333333 0.41666667 0.41666667 ... 0.58333333 0.58333333 0.66666667]' has dtype 

Epoch 1/100


2024-07-15 18:16:25.569628: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 18:16:25.570912: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 18:16:25.571701: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

393/394 [============================>.] - ETA: 0s - loss: 6918.9473

2024-07-15 18:16:39.612147: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-07-15 18:16:39.612860: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-07-15 18:16:39.613547: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

394/394 [==============================] - 15s 35ms/step - loss: 6916.4214 - val_loss: 9236.9424
Epoch 2/100
394/394 [==============================] - 13s 33ms/step - loss: 4958.1934 - val_loss: 7276.6958
Epoch 3/100
394/394 [==============================] - 13s 33ms/step - loss: 3870.0330 - val_loss: 6090.5288
Epoch 4/100
394/394 [==============================] - 13s 33ms/step - loss: 3320.2339 - val_loss: 5420.7222
Epoch 5/100
394/394 [==============================] - 13s 34ms/step - loss: 3086.9021 - val_loss: 5062.1953
Epoch 6/100
394/394 [==============================] - 14s 36ms/step - loss: 3001.5452 - val_loss: 4893.6514
Epoch 7/100
394/394 [==============================] - 15s 38ms/step - loss: 2977.7981 - val_loss: 4814.5059
Epoch 8/100
394/394 [==============================] - 14s 37ms/step - loss: 2967.7327 - val_loss: 4777.1250
Epoch 9/100
394/394 [==============================] - 15s 37ms/step - loss: 2976.0874 - val_loss: 4757.4072
Epoch 10/100
394/394 [=========